In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

from matplotlib import pyplot as plt

In [3]:
import pandas_profiling as pp

In [24]:
data = pd.read_csv('taxi.csv')
data.head(10)

,offer_gk,weekday_key,hour_key,driver_gk,order_gk,driver_latitude,driver_longitude,origin_order_latitude,origin_order_longitude,distance_km,duration_min,offer_class_group,ride_type_desc,driver_response
0,1105373,5,20,6080,174182,55.818842,37.334562,55.814567,37.355010,-1.000,-1.000,Economy,private,0
1,759733,5,14,6080,358774,55.805342,37.515023,55.819329,37.466398,18.802,25.217,Standard,private,1
2,416977,6,14,6080,866260,55.813978,37.347688,55.814827,37.354074,6.747,9.800,Economy,private,0
3,889660,2,6,6080,163522,55.745922,37.421748,55.743469,37.431130,-1.000,-1.000,Economy,private,1
4,1120055,4,16,6080,506710,55.803578,37.521602,55.812559,37.527407,12.383,19.250,Economy,private,1
5,1058203,6,15,6080,360328,55.818963,37.361033,55.814827,37.354074,6.747,9.800,Economy,private,0
6,103326,2,11,6080,615584,55.753508,37.663742,55.757251,37.659064,-1.000,-1.000,Standard,business,1
7,128861,5,20,6080,254374,55.839318,37.248862,55.839134,37.304101,1.517,3.933,Economy,private,0
8,493006,5,20,6080,753429,55.818748,37.338240,55.835764,37.294710,6.222,15.433,Economy,private,0
9,953983,6,7,6080,533820,55.839155,37.251947,55.833198,37.312197,10.410,23.083,Economy,private,0


Таргет - driver_response

Здесь сразу видим бесполнезные для нас колонки - offer_gk, driver_gk, order_gk. Это просто ID, которые в расчетах никакого смысла иметь не будут

weekday_key, hour_key - показатели дня недели и времени. Их необходимо преобразовать в категории, т.к. их абсолютное значение не должно влиять на расчет. Утро понедельника и утро субботы различаются друг от друга, хоть и часы будут одинаковы.

In [25]:
data = data.drop(['offer_gk'], axis=1)
data = data.drop(['driver_gk'], axis=1)
data = data.drop(['order_gk'], axis=1)

In [26]:
data[data['duration_min'] == -1].count()

weekday_key               26207
hour_key                  26207
driver_latitude           26207
driver_longitude          26207
origin_order_latitude     26207
origin_order_longitude    26207
distance_km               26207
duration_min              26207
offer_class_group         26207
ride_type_desc            26207
driver_response           26207
dtype: int64

In [27]:
data[data['distance_km'] == -1].count()

weekday_key               26207
hour_key                  26207
driver_latitude           26207
driver_longitude          26207
origin_order_latitude     26207
origin_order_longitude    26207
distance_km               26207
duration_min              26207
offer_class_group         26207
ride_type_desc            26207
driver_response           26207
dtype: int64

In [48]:
data[(data['distance_km'] == -1) & (data['duration_min'] == -1) & (data['driver_response'] == 1)].count()

weekday_key               14434
hour_key                  14434
driver_latitude           14434
driver_longitude          14434
origin_order_latitude     14434
origin_order_longitude    14434
distance_km               14434
duration_min              14434
offer_class_group         14434
ride_type_desc            14434
driver_response           14434
dtype: int64

В ~25% данных нет дальности и длительности поездки. Удалять их нельзя, т.к. будут большие потери данных. Вероятное решение - рассчитать их по координатам

In [28]:
from geopy.distance import vincenty

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 11 columns):
weekday_key               100000 non-null int64
hour_key                  100000 non-null int64
driver_latitude           100000 non-null float64
driver_longitude          100000 non-null float64
origin_order_latitude     100000 non-null float64
origin_order_longitude    100000 non-null float64
distance_km               100000 non-null float64
duration_min              100000 non-null float64
offer_class_group         100000 non-null object
ride_type_desc            100000 non-null object
driver_response           100000 non-null int64
dtypes: float64(6), int64(3), object(2)
memory usage: 8.4+ MB


In [30]:
data.describe()

,weekday_key,hour_key,driver_latitude,driver_longitude,origin_order_latitude,origin_order_longitude,distance_km,duration_min,driver_response
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,3.159860,13.827790,55.700824,37.492292,55.772440,37.541637,18.393518,23.134841,0.492480
std,2.054222,7.306496,2.135903,1.454640,0.665985,0.467758,41.129296,33.890105,0.499946
min,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
25%,1.000000,8.000000,55.708651,37.413497,55.708032,37.407117,-1.000000,-1.000000,0.000000
50%,3.000000,16.000000,55.772904,37.533664,55.772561,37.532537,10.411500,19.700000,0.000000
75%,5.000000,20.000000,55.960667,37.624113,55.962000,37.624358,30.665250,37.867000,1.000000
max,6.000000,23.000000,56.174097,38.280547,56.311121,38.243608,5234.283000,3958.283000,1.000000


In [31]:
#data.profile_report() --на домашнем ноутбуке не может загрузить очень долгое время (пока не разобрался, почему)

Здесь опишу несколько функций

get_one_hot - это OneHot кодирование для необходимых признаков

process_hour - группируем часы на части дня

process_day - группируем дни на будни и выходные

compute_dist_in_km - вычислим расстояние между координатами

compute_taxi_cost - сделаем собственный расчет стоимости для всех данных


In [32]:
def get_one_hot(df, cols):
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
    return df

In [33]:
def process_hour(hour):
    if 6 <= hour < 12:
        return 0
    elif 12 <= hour < 18:
        return 1
    elif 18 <= hour < 24:
        return 2
    elif 0 <= hour < 6:
        return 3

In [34]:
def process_day(day):
    if day <= 4:
        return 0
    else:
        return 1

In [35]:
def compute_dist_in_km(coords1, coords2):
    return vincenty(coords1, coords2).kilometers

In [36]:
def compute_taxi_cost(row):
    if row.offer_class_group == "Economy":
        return 97 + 7 * row.distance_km + 7 * row.duration_min
    elif row.offer_class_group == "Standard":
        return 150 + 11 * row.distance_km + 13 * row.duration_min
    elif row.offer_class_group == "Premium":
        return 299 + 15 * row.distance_km + 20 * row.duration_min
    elif row.offer_class_group == "Kids":
        return 99 + 22 * row.duration_min
    elif row.offer_class_group == "VIP":
        return 699 + 55 * row.distance_km + 25 * row.duration_min
    elif row.offer_class_group == "XL":
        return 299 + 15 * row.distance_km + 25 * row.duration_min
    else:
        return -1

In [37]:
def build_new_columns(data):
    #создание пары координат
    origin_coordindates = data[['origin_order_latitude', 'origin_order_longitude']
    ].apply(lambda x: str(x[0]) + ' ' + str(x[1]), axis=1)
    
    driver_coordinates = data[['driver_latitude', 'driver_longitude']
    ].apply(lambda x: str(x[0]) + ' ' + str(x[1]), axis=1)
    
    data['origin_coordinates'] = origin_coordindates
    data['driver_coordinates'] = driver_coordinates
       
    #get one hot
    data = get_one_hot(
        data,
        ['offer_class_group', 'ride_type_desc']
    )
    
    
    #часть дня - часть недели
    data['day_part'] = data.apply(lambda row: process_hour(row.hour_key), axis=1)
    data['week_part'] = data.apply(lambda row: process_day(row.hour_key), axis=1)
    
    #расстрояние грубо
    data['driver_to_offer_dist'] = data.apply(lambda row: compute_dist_in_km(
            (row.driver_latitude, row.driver_longitude),
            (row.origin_order_latitude, row.origin_order_longitude)
        ), axis=1)
    
    #цена поездки
    data['taxi_cost'] = data.apply(lambda row: compute_taxi_cost(row), axis=1)
    
    #удаляем лишнее - мы их уже закодировали
    data = data.drop(
        labels=['offer_class_group', 'ride_type_desc', 'weekday_key', 'hour_key'],
        axis=1
    )
     
    data['duration_min_rounded'] = data['duration_min'].apply(lambda x: np.round(x, 0))
    
    data['is_address_submitted'] = data['duration_min'].apply(lambda x: x != -1)
      
    return data

In [38]:
from sklearn.model_selection import train_test_split
y = data['driver_response'].copy()
data_train, data_test, y_train, y_test = train_test_split(data, y, test_size=0.3, random_state=42)

In [39]:
data_train = build_new_columns(data_train)
data_test = build_new_columns(data_test)

C:\Users\lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, w

In [40]:
data_train.head(10)

,driver_latitude,driver_longitude,origin_order_latitude,origin_order_longitude,distance_km,duration_min,driver_response,origin_coordinates,driver_coordinates,offer_class_group_Delivery,...,ride_type_desc_SMB,ride_type_desc_affiliate,ride_type_desc_business,ride_type_desc_private,day_part,week_part,driver_to_offer_dist,taxi_cost,duration_min_rounded,is_address_submitted
76513,55.756670,37.605210,55.741996,37.615415,11.573,31.767,1,55.74199599999999 37.61541500000001,55.75666999999999 37.60521,0,...,0,0,0,1,3,0,1.754950,1107.935,32.0,True
60406,55.961527,37.417454,55.966694,37.415754,41.589,40.750,0,55.966694 37.415754,55.961527 37.417454,0,...,0,0,0,1,2,1,0.585014,1137.229,41.0,True
27322,55.961857,37.414049,55.962322,37.407117,41.446,44.517,0,55.962322 37.407117,55.961857 37.414049,0,...,0,0,0,1,2,1,0.436015,698.741,45.0,True
53699,55.759718,37.572171,55.753287,37.584096,4.008,10.417,0,55.753287 37.584096,55.75971800000001 37.572171,0,...,0,0,0,1,1,1,1.035968,197.975,10.0,True
65412,55.763485,37.559820,55.759126,37.548757,-1.000,-1.000,0,55.759126 37.548757,55.763485 37.55982,0,...,0,0,0,1,2,1,0.847269,126.000,-1.0,False
14297,55.755313,37.613295,55.706066,37.597452,7.251,14.517,0,55.706066 37.597452,55.755313 37.613295,0,...,0,0,0,1,3,0,5.572622,770.690,15.0,True
15664,55.796050,37.378758,55.804749,37.329434,14.325,20.183,1,55.804749 37.329434,55.79605 37.37875800000001,0,...,0,0,0,1,0,1,3.241385,569.954,20.0,True
49235,55.896235,37.389717,55.888916,37.393274,2.626,6.183,1,55.888916 37.393274,55.896235 37.389717,0,...,0,0,0,1,2,1,0.844739,158.663,6.0,True
99277,55.961260,37.413497,55.962322,37.407117,42.530,48.217,1,55.962322 37.407117,55.96126 37.413497,0,...,0,0,0,1,1,1,0.415633,1244.651,48.0,True
13809,55.847282,37.626522,55.852480,37.617671,7.973,20.950,0,55.85248000000001 37.617671,55.84728199999999 37.626522,0,...,0,0,0,1,2,1,0.801420,299.461,21.0,True


Опишем колонки, которые не будут участвовать в работе модели

In [41]:
columnsForModel = [
    col for col in data_train.columns.tolist()
    if col not in set(
        [
            'driver_response',
            'origin_coordinates',
            'driver_coordinates',
            'duration_min_rounded',
        ]
    )
]

In [42]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [43]:
model = LinearRegression()
model.fit(data_train[columnsForModel], y_train) # Обучение

y_pred = model.predict(data_test[columnsForModel])

print("R2: \t", r2_score(y_test, y_pred))
print("RMSE: \t", np.sqrt(mean_squared_error(y_test, y_pred)))
print("MAE: \t", mean_absolute_error(y_test, y_pred))

R2: 	 0.13327425330493503
RMSE: 	 0.4653845963763353
MAE: 	 0.43578971035541203


На данный момент модель не дала особых результатов.
Необходимо доработать.